# dqRunning
### quick_tutorial

__O objetivo desse tutorial é apresentar de maneira rápida as principais funcionalidades da biblioteca dataquality_bnr__. A biblioteca traz diversos módulos que tornam os processos necessários à validacao qualitativa dos dados/tableas uma tarefa muito mais simples, intuitiva e automatizada.

### Installation

O primeiro passo é fazer a instalacao do pacote via *pip install*

In [1]:
!pip install dataquality_bnr

Looking in indexes: http://artifactory.produbanbr.corp/artifactory/api/pypi/pypi-all/simple
     |████████████████████████████████| 1.6 MB 4.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/miniconda/bin/python3 -m pip install --upgrade pip' command.


### Set up a PySpark session
A biblioteca foi construida para ser utilizada com __PySpark__ e possibilitar '*testes unitarios dos dados*', executando validacoes qualitativas em datasets de larga escala.</br>
A integracao da sessao spark e a biblioteca depende apenas de duas configuracoes adicionais:


In [1]:
from pyspark.sql import SparkSession, Row
from dataquality_bnr.dqSupport import main as dqSup

spark = SparkSession\
        .builder\
        .config("spark.jars", dqSup.getDeequJar_path())\
        .config("spark.jars.excludes", dqSup.getDeequJar_excludes())\
        .getOrCreate()

### Defining dataquality views 
A partir da definicao de tres arquivos .yaml é feita toda a etruturacao necessaria ao processo de dataquality.
* Em *inputData* é definida qual a tabela de entrada.
* Em *infraYaml* sao definidas particularidades da infraestrutura do dataquality.
* Em *vsYaml* sao definidas todas as vericacoes a serem feitas na tabela de entrada.

In [7]:
dqView_simple = {"viewName" : "dqView_check_moderate",
                 "inputData": "yamlFiles/quick_tutorial/a_check/inputData.yaml",
                 "infraYaml": "yamlFiles/quick_tutorial/a_check/infrastructure.yaml",
                 "vsYaml": "yamlFiles/quick_tutorial/a_check/verificationSuite.yaml"}

### Set up & run()
Com a estrutura da 'view' já definida, é necessária agora, apenas a __atribuicao de um diretório raiz__.<br>
O diretório informado servira como um espaco disponivel a essa instancia especifica do dataquality, onde serao armazenadas variaveis internas e também serao apresentados resultados, como por exemplo, arquivos .parquet e .csv resultantes das execusoes.


In [ ]:
from dataquality_bnr.dqRunning import main as dqRun

myDq = dqRun.Dq(spark, "/user/x266727/dataquality_bnr-docs/quick_tutorial/DQ/")

myDq = (myDq
        .addView(dqView_simple, "Check"))

dq_run_return = myDq.run()

### dq_run_return
Todos os resultados estao armazenados na estrutura interna de diretórios do seu processo de dataquality, e também, nesse objeto de retorno 'dq_run_return'.
Interagindo com esse objeto podemos carregar os resultados obtidos utilizando dois principais métodos:
* Com *get_overall_booleanResult* temos uma váriavel True ou False da aprovacao ou nao aprovacao da tabela de entrada.
* Com *get_overall_csvResult_df* temos um dataframe com a listagem de todos os possíveis Erros ou Alertas encontrados em validacoes feitas na tabela de entrada

In [18]:
dq_run_return.get_overall_booleanResult()

True

In [19]:
csvResult_df = dq_run_return.get_overall_csvResult_df(spark)
csvResult_df.toPandas()

,check,check_level,check_status,constraint,constraint_status,constraint_message,dataset_date,YY_MM_DD,viewName,viewPath
0,None,None,Success,None,None,None,None,None,overall,None


#### pydeequ shutdown_callback_server()
#### spark.stop()
__Importante!__
Após a execucao dos jobs, garanta que a sessao __spark__ juntamente com o __callback_server__ sejam encerrados, evitando que qualquer processo "fantasma" fique pendurado.<br>
Leia mais sobre __Pydeequ__ e __callback_server__ em: https://github.com/awslabs/python-deequ

In [ ]:
spark.sparkContext._gateway.shutdown_callback_server()
spark.stop()